In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('D:/Potato_Disease/potatoes.h5')

# Use the loaded model for predictions

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

# Load your pre-trained model
model_path = 'D:/Potato_Disease/potatoes.h5'  # Replace with the actual path to your .h5 file
model = tf.keras.models.load_model(model_path)

# Load and preprocess the input image
image_path = "D:/pot2.jpeg"  # Replace with the actual path to your input image
img = image.load_img(image_path, target_size=(256, 256))  # Adjust target_size according to your model's input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
predictions = model.predict(img_array)

predicted_class = np.argmax(predictions, axis=1)

# Print the predicted class
print(f"Predicted Class: {predicted_class}")


1/1 [==============================] - 0s 94ms/step
Predicted Class: [0]


In [14]:
predictions

array([[1.000000e+00, 2.441478e-25, 0.000000e+00]], dtype=float32)

In [12]:
class_dict={"Early_Blight": 0, "Late_Blight": 1,"Healthy":2}
import json
with open("D://Potato_Disease//class_dictionary.json","w") as f:
    f.write(json.dumps(class_dict))

In [1]:
import cv2

import numpy as np
import base64
import json

import tensorflow as tf
from tensorflow.keras.models import load_model

class_dict={"Early_Blight": 0, "Late_Blight": 1,"Healthy":2}

__model=None
def get_cv2_image_from_base64_string(b64str):
    encoded_data=b64str.split(',')[1]
    nparr=np.frombuffer(base64.b64decode(encoded_data),np.uint8)
    img=cv2.imdecode(nparr,cv2.IMREAD_COLOR)
    
    return img



def classify_image(image_base64_data, file_path=None):
    imgs = cropped_image(file_path, image_base64_data)

    result = []
    x=[]
    for img in imgs:
        
        
        # Resize the image if necessary
        resized_image = cv2.resize(img, (48, 48))

        # Normalize the pixel values if necessary
        normalized_image = resized_image / 255.0

        # Reshape the image to match the expected input shape
        reshaped_image = normalized_image.reshape((1, 48, 48, 1))

        # Make predictions
        predictions = __model.predict(reshaped_image)
        print(predictions)
        predicted_class = predictions.argmax()
        for i in class_dict:
            if(class_dict[i]==predicted_class):
                predicted_text=i
                break
        

        # Get the probability scores for each class
        print(predictions)
        probability_scores = predictions[0]
        class_probabilities = [round(float(score),5)*100 for score in probability_scores]
        
        
        result.append({
            'class': predicted_text,
            'class_probability': class_probabilities,
            'class_dictionary': __class_name_to_number
        })
        
        

    return result
    
def class_number_to_name(class_num):
    return __class_number_to_name[class_num]
        

def load_saved_artifacts():
    print("Loading saved Artifacts...")
    
    global __class_name_to_number
    global __class_number_to_name
    with open('D://Potato_Disease//class_dictionary.json',"r") as f:
        __class_name_to_number=json.load(f)
        __class_number_to_name={v:k for k,v in __class_name_to_number.items()}
    global __model
    if __model is None:
        __model=load_model('D://Potato_Disease//POTATO.h5', compile=False)
    print("Loading Artifacts Done")
    

def cropped_image(image_path,image_base64_data):
    if image_path:
        img=cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img=get_cv2_image_from_base64_string(image_base64_data)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        
        

    print(img)
    return [img]
     

def get_b64_test_image():
    with open("D:\\MLProject\\b64.txt") as f:
        return f.read()
    
if __name__=="__main__":
    load_saved_artifacts()
    print(classify_image(get_b64_test_image(),None))


Loading saved Artifacts...
Loading Artifacts Done
[[156 184 198 ... 157 154 150]
 [146 182 199 ... 161 154 150]
 [135 176 195 ... 165 161 155]
 ...
 [ 28  25  21 ... 179 175 173]
 [ 29  18  22 ... 177 172 169]
 [ 21  14  23 ... 172 167 161]]
1/1 [==============================] - 0s 344ms/step
[[0.12080115 0.7258798  0.153319  ]]
[[0.12080115 0.7258798  0.153319  ]]
[{'class': 'Late_Blight', 'class_probability': [12.08, 72.588, 15.332], 'class_dictionary': {'Early_Blight': 0, 'Late_Blight': 1, 'Healthy': 2}}]


In [2]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/classify',methods=['GET','POST'])
def classify():
    image_data = request.form['image_data']
    results = classify_image(image_data)
    print("result : ",results)
    response = jsonify(results)
    response.headers.add('Access-Control-Allow-Origin', '*')
    print(response)
    return response
    
    
    

if __name__ == "__main__":
    print("Staring Python Flask Server.....")
    load_saved_artifacts()

    app.run(port=5000)

Staring Python Flask Server.....
Loading saved Artifacts...
Loading Artifacts Done
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


[[183 183 183 ... 207 141 136]
 [184 183 183 ... 144 171 166]
 [182 182 181 ... 177 156 178]
 ...
 [137 142 146 ...  45  41  49]
 [138 141 143 ...  42  45  44]
 [140 140 140 ...  42  48  40]]
1/1 [==============================] - 0s 41ms/step


127.0.0.1 - - [21/Dec/2023 19:59:02] "POST /classify HTTP/1.1" 200 -


[[0.12211379 0.7232494  0.15463679]]
[[0.12211379 0.7232494  0.15463679]]
result :  [{'class': 'Late_Blight', 'class_probability': [12.211, 72.32499999999999, 15.464], 'class_dictionary': {'Early_Blight': 0, 'Late_Blight': 1, 'Healthy': 2}}]
<Response 146 bytes [200 OK]>
